## FIFA Ranking Predictor

### Initialization

In [1]:
%matplotlib inline

from azureml import Workspace
import pandas as pd
import numpy as np

ws = Workspace()
ds = ws.datasets['fifa_ranking.csv']
fifa_df= ds.to_dataframe()

## Converting columns to numeric values
cols = ['rank_date']
fifa_df[cols] = fifa_df[cols].apply(pd.to_numeric, args=('coerce',))


##print (fifa_df.dtypes)
##fifa_df.head(5)

/home/nbuser/anaconda3_23/lib/python3.4/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/nbuser/anaconda3_23/lib/python3.4/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Random Sampling

In [2]:
fifa_sam = fifa_df.sample(40000)

### Feature selection

In [3]:
keep_col = [
    "total_points",
    "last_year_avg_weighted",
    "two_year_ago_weighted",
    "three_year_ago_weighted",
    "rank_date", "rank",
     ]

fifa_sam = fifa_sam[keep_col]

fifa_sam.columns= ["total_points",
    "last_year_avg_weighted",
    "two_year_ago_weighted",
    "three_year_ago_weighted",
    "rank_date","rank",]

In [4]:
fifa_sam.head(5)

,total_points,last_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,rank_date,rank
35505,0.00,0.00,0.00,0.00,2009,94
1052,0.00,0.00,0.00,0.00,1994,49
28012,0.00,0.00,0.00,0.00,2006,49
46514,538.52,112.09,64.13,24.96,2014,66
31145,0.00,0.00,0.00,0.00,2007,82


### Forming y Matrix


In [5]:
y = fifa_sam["rank"]

fifa_sam = fifa_sam.drop(labels=["rank"], axis=1)

y = np.reshape (y, (40000,1))


print(y.shape)


(40000, 1)


/home/nbuser/anaconda3_23/lib/python3.4/site-packages/numpy/core/fromnumeric.py:224: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return reshape(newshape, order=order)


### Theta Matrix

In [6]:
m = len(fifa_sam)
theta = pd.DataFrame(columns=["theta"])

for index, value in enumerate(fifa_sam.columns.values):
    theta.loc[index] = 0

fifa_sam = fifa_sam.astype(np.float64)

In [12]:
print(theta.shape)
print(fifa_sam.shape)

fifa_sam.head(5)

(5, 1)
(40000, 5)


,total_points,last_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,rank_date
35505,0.00,0.00,0.00,0.00,2009.0
1052,0.00,0.00,0.00,0.00,1994.0
28012,0.00,0.00,0.00,0.00,2006.0
46514,538.52,112.09,64.13,24.96,2014.0
31145,0.00,0.00,0.00,0.00,2007.0


### Normal Equation

In [13]:
def compute_normal(fifa_sam, y):
    norm_T = fifa_sam.T
    prod_df = norm_T.dot(fifa_sam)
    inv_df = pd.DataFrame(np.linalg.pinv(prod_df.values))
    new_df = inv_df.dot(norm_T.values)
    theta = new_df.dot(y)
    return theta
normal_theta = compute_normal(fifa_sam, y)

In [14]:
normal_theta

,0
0,-0.099222
1,0.034275
2,0.047278
3,0.058096
4,0.055340


### Cost Function

In [18]:
def cost_function(theta, X, y, reg_param):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    m = y.shape[0]
    h = X * theta
    error = np.power((h - y), 2)
    error = np.sum(error)
    term = error / (2*m)
    reg = (reg_param * np.sum(np.power(theta[1:, :], 2))) / (2*m)

    return term + reg


In [21]:
cost_function(normal_theta, fifa_sam, y, m)

1497.0082744833726